# Aleasim Developer Guide
* Author : Yousif El-Wishahy (yousif.elwishahy@ubcorbit.com)
* Date: 2024-08-06
* Aleasim Design Document: https://docs.google.com/document/d/13W0VLFCf4jPUaaTYP6YeOesYwN5_omOXdhjFnd8Qf2M/edit

# Aleasim Basics
The following guide and code cells assume you have configured the developer environment properly and have all packages installed.
Ensure that the interpreter of this Jupyter notebook is set to the aleasim poetry environment.

### Kernel and Model Creation

In [ ]:
#IMPORTS
#import the aleasim package
import alea.sim

#alternatively (reccomended), import specific modules from the package
from alea.sim.kernel.kernel import AleasimKernel

The aleasim.kernel.kernel.AleasimKernel class is responsible for startup/shutdown, connecting all the models, advancing the simulation time, and providing utility classes. The kernel manages simulation time and holds references to the Scheduler and FrameManager classes which handle simulation events and spatial transformations. 

Most importantly, the kernel holds a reference to “root” which is the root model of the simulation. The root model can be any class that implements AbstractModel, however, for simplicity the root model does nothing but contain other models. A class named AleasimRootModel implements AbstractModel.

In [ ]:
#A simulation that has no models

kernel = AleasimKernel()

We just instantiated a kernel. Let's add some models.

In [ ]:
#adding attitude dynamics to the kernel
from alea.sim.epa.attitude_dynamics import AttitudeDynamicsModel
from alea.sim.epa.orbit_dynamics import OrbitDynamicsModel

#first create the model object
#you must give it a name, and pass a reference of the kernel to it
adyn: AttitudeDynamicsModel = AttitudeDynamicsModel(kernel)

#at this point, the models __init__() and configure() functions should have been called.

#next you can add the model to the kernel
#you can optionally specify to create memory for it to store its state data in
kernel.add_model(adyn, create_shared_mem=True)

#now add orbital dynamics
odyn = OrbitDynamicsModel(kernel)
kernel.add_model(odyn, create_shared_mem=True)

kernel.step()


A couple of things just happened after we added an AttitudeDynamicsModel to the simulation:
* Attitude dynamics and orbital dynamics loaded configurations from files in `aleasim/aleasim/configuration/` , if you ever create a new model you should be adding configuration files in there
* Shared memory was created in attitude dynamics

FrameManager is a class that handles reference frame updates, it requires knowledge from orbit dynamics and attitude dynamics to get information for the transformations.

### Logging

Now that orbit dynamics is added, the frame manager is happily logging all the frame transformations between the different frames.

Sometimes logs can be excessive, logging level can be set on a model by model basis.

In [ ]:
import logging

#set logging levels for different components
kernel.frame_manager.logger.setLevel(logging.INFO)
adyn.logger.setLevel(logging.DEBUG)
odyn.logger.setLevel(logging.DEBUG)

kernel.step()

We are still getting debug info for the dynamics models, but the frame manager is now silent.

In [ ]:
print('we should see no prints after this')
adyn.logger.setLevel(logging.INFO)
odyn.logger.setLevel(logging.INFO)
kernel.step()


adyn.logger.setLevel(logging.DEBUG)
odyn.logger.setLevel(logging.DEBUG)
print('we should see some prints after this')
kernel.step()

adyn.logger.setLevel(logging.INFO)
odyn.logger.setLevel(logging.INFO)

### The Scheduler and Events

The Scheduler (aleasim.kernel.scheduler.Scheduler) is a generic interface for scheduling events. An instance of the scheduler is managed by the AleasimKernel and all children models register events to it.

Each update event that was logged previously is a registered event in the scheduler. When we call kernel.step() the schedule executes all events at that timestep. 

An event in this context is a Dataclass containing some metadata: (time, priority, sequence, action, period, argument ).
time - simulator time to execute event at
priority - events scheduled for the same time will be executed in the order of their priority
sequence - A continually increasing sequence number that  separates events if time and priority are equal
period - Time offset for next periodic event, if -1 will assume not periodic.
action - Executing the event means executing action(*argument, **kwargs)
argument -  a sequence holding the positional arguments for the action.

At its core the scheduler maintains a queue of ordered events (based on time -> priority -> sequence number). The key point is that “Executing the event means executing action(*argument, **kwargs)”. The general way that events should be used in the simulation is to schedule periodic events to the update function of the model, or schedule one-time events when needed based on logic. The scheduler is permitted to run while events can be added to the queue. For now this scheduler is designed to run in a single-threaded, single-process manner, but in general can support multiple threads.

Lets look at the pending events!

In [ ]:
#print the scheduler event queue
print(f'length of event queue is {len(kernel.scheduler.queue)}')
print(kernel.scheduler.queue)

You should see that the next event in the queue `Event(time=6, priority=10, is_periodic=True, period=1, action=<bound method FrameManager.update_frames of <aleasim.kernel.frame_manager.FrameManager object at 0xsomeRamLocation>>`

But we can add more events:


In [ ]:
def test_event_function():
    print("test event!")

#schedule an function event to execute immediately with a priority of 1, and only execute once
kernel.schedule_event(delay=0, priority=1, action=test_event_function)

#we should see a printout now
kernel.step()


### Simulation Time

In [ ]:
#we can call kernel.advance to advance simulation time for x seconds

#advance for 0.1 seconds
kernel.advance(0.1)

#simulation timestep
print(f'simulation timestep is {kernel.timestep} s')

#simulation seconds since start
print(f'simualtion time in seconds is {kernel.time}')

#the float time has floating point error
#internally, integers are used
print(f'simulation integer time is {kernel._t_n} (multiples of timestep)')

#simulation date in utc
print(f'simulation date is {kernel.date}')

#skyfield epoch time is used by the simulation for computing planet locations
print(f'simulation skyfield epoch time is {kernel.skyfield_time}')


## Model Development Example

Let's take a quick look at creating a basic dummy magnetic sensor model.

We want it to do 2 things
* power on/off and consume power
* report magnetic field value with some constant bias

Let's grab a section from the design doc:

#### Abstract Model
The core Aleasim model is the AbstractModel which has some generic functionalities that will be inherited by all models. This functionality includes generic functions for logging, configuring and connecting models, updating the model state, and containing children models. All AbstractModels also hold a reference to the kernel and scheduler.


Methods that all inheriting classes must implement:
* connect
connect models to each other, and obtain references to attributes of other models. This function may return an error if the parent is undefined or if a model is missing from the tree. A design choice was made for the kernel to be responsible for calling this method.

Properties that all inheriting classes must implement:
* config_name
File name prefix used to search for config files.

#### Dynamic Model
Inherits AbstractModel, contains functionality for linear or nonlinear state update function and numerical integration of the state update functions

#### Powered Unit Model
Functionality for a powered unit that turns on/off and consumes power.


In [ ]:
#based on the docs, we want to 
# 1. inherit powered unit which inherits abstract model
# 3. obtain magnetic field data somehow
# 4. write a sensor output function

#first import abstract model
from alea.sim.kernel.generic.abstract_model import AbstractModel
from alea.sim.kernel.generic.powered_unit_model import PoweredUnitModel

#magnetic field model
from alea.sim.epa.magnetic_field_model import EarthMagneticFieldModel
from alea.sim.kernel.kernel import AleasimKernel

#frame imports
from alea.sim.kernel.frames import ReferenceFrame
from alea.sim.kernel.frame_manager import FrameManager

#numpy for vector math
import numpy as np

from math import degrees
class MagSensorModel(PoweredUnitModel):
    """
    Simple mag sensor that returns the local magnetic field to the spacecraft with some bias added to it.
    """   
    
    def __init__(self, name: str, sim_kernel: AleasimKernel, power_usage: float = 0) -> None:
        super().__init__(name, sim_kernel, power_usage)
        
        #sensor frame
        #use the sim body frame as the sensor frame
        self._frame_manager: FrameManager = self.kernel._frame_manager
        self._frame = self._frame_manager.body_frame
        
        self._constant_bias: np.ndarray = np.ones(3)*2.73e-8
        
        #magnetic field ground truth model
        #remember to grab models in connect()
        self._magm: EarthMagneticFieldModel = None
        
        #we also need spacecraft coordinates for magnetic field value
        self._odyn : OrbitDynamicsModel = None

    @property
    def config_name(self) -> str:
        #dummy value for now
        return 'magnetic_sensor'

    def connect(self):
        #grab the magnetic field model
        self._magm = self.kernel.get_model(EarthMagneticFieldModel)
        self._odyn = self.kernel.get_model(OrbitDynamicsModel)
    
    #custom function
    def measure(self) -> np.ndarray:
        #get spacecraft lla coords [lon radians, lat radians, altitude  km]
        lla = odyn.position_lonlat
        #convert lon and lat to degrees from radians
        lla[0] = degrees(lla[0])
        lla[1] = degrees(lla[1])

        #get the magnetic field vector in NED frame
        mag_ned = self._magm.get_mag_vector(lla)
        
        #transform from ned to desired frame
        mag_sens = self._frame.transform_vector_from_frame(mag_ned, self._frame_manager.ned_frame)
        
        #add a contant bias to it
        mag_sens += self._constant_bias
        
        return mag_sens


We now have a basic sensor class `MagSensorModel` lets try to add it to the simulation.

In [ ]:
magsense = MagSensorModel('mag_sensor', kernel, power_usage=1e-3)

kernel.add_model(magsense)

Let's try to step the simulation and measure the sensor.

In [ ]:
try:
    kernel.step()
except Exception as errmsg:
    print(errmsg)

Aha! We forgot to do one thing... `MagSensorModel` searches for `EarthMagneticFieldModel` but the simulation has no such model ... yet.

In [ ]:
#add magnetic field model
magm = EarthMagneticFieldModel(kernel)
kernel.add_model(magm)

#force call connect again to find the magnetic model
magsense.connect()

#now try stepping
kernel.step()

#print a measurement
print(f'**********\nmag sensor measurement: {magsense.measure()} nT\n**********')

In [ ]:
print(f'spacecraft position in eci [m] is {odyn.position_eci}')

Note that we haven't used any of the power logic yet. If we like we could allow it so that the measurement is only valid if the unit is powered on. I leave at as an exercise for the reader. For now here's an example of the inherited power functions.

In [ ]:
print(magsense.powered_state)

magsense.power_on()

print(magsense.current_power_usage)
print(magsense._power_on_time)

### Shared Memory Implementation

If we want to plot model data, we must first implement it as a shared memory model using the `SharedMemoryModelInterface`.

As of aleasimv0.1, you only need to implement two attributes of `SharedMemoryModelInterface`:
```python
    @property
    @abc.abstractmethod
    def saved_state_size(self) -> int:
        """
        This method must be implemented by inheriting subclass.
        length of a single chunk for model datastorage, typically composed of float64s
        this information is relevant to shared memory
        """
        raise NotImplementedError

    @property
    @abc.abstractmethod
    def saved_state_element_names(self) -> list[str]:
        """
        This method must be implemented by inheriting subclass.
        list of names for each element in the saved state
        """
        raise NotImplementedError
```

Let's look at an implementation of the sensor but with a shared memory feature. Pay attention the the `saved_state_size` and `saved_state_element_names` properties and the `measure` function, as only those need to change.

In [ ]:
#import SharedMemoryModelInterface
from alea.sim.kernel.kernel import SharedMemoryModelInterface

#lets redfined our sensor class
#now it inerits both PoweredUnitModel and SharedMemoryModelInterface
class MagSensorModelWithMemory(PoweredUnitModel, SharedMemoryModelInterface):
    """
    Simple mag sensor that returns the local magnetic field to the spacecraft with some bias added to it.
    """   
    
    def __init__(self, name: str, sim_kernel: AleasimKernel, power_usage: float = 0) -> None:
        super().__init__(name, sim_kernel, power_usage)
        
        #sensor frame
        #use the sim body frame as the sensor frame
        self._frame_manager: FrameManager = self.kernel._frame_manager
        self._frame = self._frame_manager.body_frame
        
        self._constant_bias: np.ndarray = np.ones(3)*2.73e-8
        
        #magnetic field ground truth model
        #remember to grab models in connect()
        self._magm: EarthMagneticFieldModel = None
        
        #we also need spacecraft coordinates for magnetic field value
        self._odyn : OrbitDynamicsModel = None
        
    #implement the shared memory attribute
    @property
    def saved_state_size(self) -> int:
        #each sensor measurement we want is a vector of length 3
        return 3

    @property
    def saved_state_element_names(self) -> list[str]:
        return ['mx', 'my', 'mz']

    @property
    def config_name(self) -> str:
        #dummy value for now
        return 'magnetic_sensor'

    def connect(self):
        #grab the magnetic field model
        self._magm = self.kernel.get_model(EarthMagneticFieldModel)
        self._odyn = self.kernel.get_model(OrbitDynamicsModel)

    
    #custom function
    #MODIFIED TO SAVE A MEMORY CHUNK
    def measure(self) -> np.ndarray:
        #get spacecraft lla coords [lon radians, lat radians, altitude  km]
        lla = odyn.position_lonlat
        #convert lon and lat to degrees from radians
        lla[0] = degrees(lla[0])
        lla[1] = degrees(lla[1])

        #get the magnetic field vector in NED frame
        mag_ned = self._magm.get_mag_vector(lla)
        
        #transform from ned to desired frame
        mag_sens = self._frame.transform_vector_from_frame(mag_ned, self._frame_manager.ned_frame)
        
        #add a contant bias to it
        mag_sens += self._constant_bias
        
        #lets also save this chunk to memory now
        self.save_chunk_to_memory(mag_sens)
        
        return mag_sens


In [ ]:
#lets try add it to the kernel

magsense2 = MagSensorModelWithMemory('magsense2', kernel, power_usage=1e-3)

#you must set create_shared_mem to True, it is off by default
kernel.add_model(magsense2, create_shared_mem=True)

print(f'magsense2 shared array: {magsense2.state_array}')

kernel.step()
magsense2.measure()

print(f'magsense2 shared array after measure: {magsense2.state_array}')

## Plotting and Analysis

Now that we have created shared memory, lets look at some attitude date.

The attitude dynamics model describes the state vector which it saves to shared memory as 

```
    The spacecraft attitude state is is a 7 element vector composed of
    - quaternion elements (4) (rotation of body frame w.r.t inertial frame)
    - body angular rates (3) (angular velocity of body frame w.r.t inertial frame)
```

This gives us enough information to print some elements.

In [ ]:
import matplotlib.pyplot as plt

#first run the kernel for a few seconds
#recall the timestep is 0.001
kernel.advance(2)

#plot only 1000 values
n = 1000

objs = plt.plot(adyn.state_array[:n,0:4])
plt.title("Quaternion Elements")
plt.legend(iter(objs), ('q0', 'q1', 'q2', 'q3'))
plt.show()

objs = plt.plot(adyn.state_array[:n,4:7])
plt.title("Angular Rates")
plt.legend(iter(objs), ('w1', 'w2', 'w3'))
plt.show()

That was boring. The state vector is just constant. Let's add some angular velocity.

In [ ]:
#set a new angular velocity of body frame relative to inertial
new_state = adyn._state
new_state[4:7] = [30, 0, -10] #rad/s
adyn.set_state(new_state)

#start index
nstart = adyn.arr_size

#first run the kernel for 1 second
#recall the timestep is 0.001
kernel.advance(1)

#plot only 1000 values
n = nstart + 1000

objs = plt.plot(adyn.state_array[nstart:n,0:4])
plt.title("Quaternion Elements")
plt.legend(iter(objs), ('q0', 'q1', 'q2', 'q3'))
plt.show()

objs = plt.plot(adyn.state_array[nstart:n,4:7])
plt.title("Angular Rates")
plt.legend(iter(objs), ('w1', 'w2', 'w3'))
plt.show()

## Environment Vector: Sun Vector in ECI and BODY frames

In [ ]:
#the sun vector is obtained from orbital information , so it is contained in the orbit dynamics model

#odyn returns the sun vector in units of au

#get sun vector in eci frame
sun_vec_eci: np.ndarray = odyn.calculate_sun_vector()
print('sun vec eci', sun_vec_eci)

#convert to body frame (to use as a sensor input for example)
sun_vec_body: np.ndarray = kernel.body_frame.transform_vector_from_frame(sun_vec_eci, kernel.eci_frame)
print('sun vec body', sun_vec_body)

#they should have the same magnitude
print(f'|sun_vec_eci| == |sun_vec_body| is {np.linalg.norm(sun_vec_body) == np.linalg.norm(sun_vec_eci)}')